In [29]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#from sklearn.datasets import fetch_openml
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [2]:
%matplotlib inline

In [3]:
df = pd.read_csv('books.csv', 
                on_bad_lines='skip')
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [5]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [6]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.00,11123.00,11123.00,11123.00,11123.00,11123.00
mean,21310.86,3.93,9759880247639.15,336.41,17942.85,542.05
std,13094.73,0.35,442975846058.35,241.15,112499.15,2576.62
min,1.00,0.00,8987059752.00,0.00,0.00,0.00
25%,10277.50,3.77,9780345453803.50,192.00,104.00,9.00
50%,20287.00,3.96,9780582461536.00,299.00,745.00,47.00
75%,32104.50,4.14,9780872208045.50,416.00,5000.50,238.00
max,45641.00,5.00,9790007672386.00,6576.00,4597666.00,94265.00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


In [8]:
df.shape

(11123, 12)

In [9]:
df.sample(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
5254,18963,Morgoth's Ring (The History of Middle-Earth #10),J.R.R. Tolkien/Christopher Tolkien,4.13,0395680921,9780395680926,eng,478,1619,44,12/14/1993,Houghton Mifflin Harcourt
9572,38107,Case Closed: Lee Harvey Oswald and the Assassi...,Gerald Posner,3.96,1400034620,9781400034628,eng,640,1194,107,9/9/2003,Anchor Books
7385,28425,Intimate Enemies,Shana Abe,3.81,0553581996,9780553581997,eng,390,401,27,6/6/2000,Bantam
6324,23841,Nightmare House (Harrow House #1),Douglas Clegg,3.36,084395177X,9780843951776,en-GB,340,971,101,5/1/2004,Leisure Books
2984,11002,Eight Men Out: The Black Sox and the 1919 Worl...,Eliot Asinof/Stephen Jay Gould,4.03,0805065377,9780805065374,en-US,336,8614,242,5/1/2000,Holt Paperbacks
3116,11464,Old School,Tobias Wolff,3.82,0747574650,9780747574651,eng,196,7772,826,2/7/2005,Bloomsbury
10742,43824,Vittorio the Vampire (New Tales of the Vampire...,Anne Rice,3.52,0375401601,9780375401602,eng,292,685,36,3/8/1999,Knopf Publishing Group
806,2687,Novels 1896–1899: The Other House / The Spoils...,Henry James/Myra Jehlen,4.28,1931082308,9781931082303,eng,1035,25,4,3/10/2003,Library of America
10494,42728,Rover rettet Weihnachten.,Roddy Doyle/Brian Ajhar,4.12,3570127214,9783570127216,ger,160,1,0,9/1/2002,Omnibus Hc Bei Bertelsmann
9685,38667,Pedro Páramo,Juan Rulfo,4.05,9685208557,9789685208550,spa,132,1386,209,12/1/2009,Editorial RM


In [10]:
df.language_code.unique()

array(['eng', 'en-US', 'fre', 'spa', 'en-GB', 'mul', 'grc', 'enm',
       'en-CA', 'ger', 'jpn', 'ara', 'nl', 'zho', 'lat', 'por', 'srp',
       'ita', 'rus', 'msa', 'glg', 'wel', 'swe', 'nor', 'tur', 'gla',
       'ale'], dtype=object)

In [11]:
df.language_code.nunique()

27

In [12]:
df.sort_values('ratings_count', ascending=False).head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
10336,41865,Twilight (Twilight #1),Stephenie Meyer,3.59,0316015849,9780316015844,eng,501,4597666,94265,9/6/2006,Little Brown and Company
1697,5907,The Hobbit or There and Back Again,J.R.R. Tolkien,4.27,0618260307,9780618260300,eng,366,2530894,32871,8/15/2002,Houghton Mifflin
1462,5107,The Catcher in the Rye,J.D. Salinger,3.80,0316769177,9780316769174,eng,277,2457092,43499,1/30/2001,Back Bay Books
307,960,Angels & Demons (Robert Langdon #1),Dan Brown,3.89,1416524797,9781416524793,eng,736,2418736,21303,4/1/2006,Pocket Books
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4415,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
23,34,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.36,0618346252,9780618346257,eng,398,2128944,13670,9/5/2003,Houghton Mifflin Harcourt
2114,7613,Animal Farm,George Orwell/Boris Grabnar/Peter Škerl,3.93,0452284244,9780452284241,eng,122,2111750,29677,5/6/2003,NAL
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.


In [13]:
df.average_rating.nunique()

209

In [14]:
df.sort_values('average_rating', ascending=False).head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
624,2034,Comoediae 1: Acharenses/Equites/Nubes/Vespae/P...,Aristophanes/F.W. Hall/W.M. Geldart,5.00,0198145047,9780198145042,grc,364,0,0,2/22/1922,Oxford University Press USA
9893,39829,His Princess Devotional: A Royal Encounter Wit...,Sheri Rose Shepherd,5.00,1590529626,9781590529621,eng,240,2,0,10/16/2007,Multnomah
4788,17224,The Diamond Color Meditation: Color Pathway to...,John Diamond,5.00,1890995525,9781890995522,eng,74,5,3,2/1/2006,Square One Publishers
9324,36853,Tyrannosaurus Wrecks (Stanley #1),Laura Driscoll/Alisa Klayman-Grodsky/Eric ...,5.00,0786845031,9780786845033,eng,24,2,1,2/1/2003,Disney Press
9720,38804,The Irish Anatomist: A Study of Flann O'Brien,Keith Donohue,5.00,1930901356,9781930901353,eng,222,1,0,7/25/2003,Academica Press
4933,17830,Bulgakov's the Master and Margarita: The Text ...,Elena N. Mahlow,5.00,0533017424,9780533017423,eng,202,4,0,1/1/1975,Vantage Press
6775,25426,Delwau Duon: Peintiadau Nicholas Evans = Symph...,Nicholas Evans/Rhonda Evans,5.00,0862431352,9780862431358,wel,150,1,0,6/22/1987,Y Lolfa
9282,36492,Oliver Wendell Holmes in Paris: Medicine Theo...,William C. Dowling,5.00,1584655801,9781584655800,eng,179,1,1,2/28/2007,University Press of New England
786,2560,Willem de Kooning: Late Paintings,Julie Sylvester/David Sylvester,5.00,382960226X,9783829602266,eng,83,1,0,9/1/2006,Schirmer Mosel
4125,14741,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,5.00,0744002354,9780744002355,eng,128,2,0,3/6/2003,BradyGames


In [15]:
df['authors'].value_counts()

authors
Stephen King                                              40
P.G. Wodehouse                                            40
Rumiko Takahashi                                          39
Orson Scott Card                                          35
Agatha Christie                                           33
                                                          ..
Legs McNeil/Gillian McCain                                 1
Adam Woog                                                  1
Mikal Gilmore                                              1
Thich Nhat Hanh/Arnold Kotler/Dalai Lama XIV               1
William T. Vollmann/Larry McCaffery/Michael Hemmingson     1
Name: count, Length: 6639, dtype: int64

In [16]:
model = df.copy()

In [17]:
model.average_rating = model.average_rating.astype(float)

In [18]:
model.shape

(11123, 12)

In [19]:
model[model.average_rating <1 ].count()

bookID                25
title                 25
authors               25
average_rating        25
isbn                  25
isbn13                25
language_code         25
  num_pages           25
ratings_count         25
text_reviews_count    25
publication_date      25
publisher             25
dtype: int64

In [20]:
type(model.average_rating)

pandas.core.series.Series

In [21]:
def find_category(average_rating):
    if average_rating <= 1:
        return 'one stars'
    elif average_rating > 1 and average_rating <= 2:
        return 'two star'
    elif average_rating > 2 and average_rating <= 3:
        return 'three star'
    elif average_rating > 3 and average_rating <= 4:
        return 'four star'
    elif average_rating > 4 and average_rating <= 5:
        return 'five star'

In [22]:
print(find_category(6))

None


In [23]:
model['rating_catergory'] = model['average_rating'].apply(lambda x: find_category(x))

In [24]:
model.sample(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,rating_catergory
758,2495,The Time Machine,H.G. Wells/Cynthia Brantley Johnson/Benjamin B...,3.89,0743487737,9780743487733,eng,150,372,36,7/1/2004,Simon Schuster,four star
4984,18022,Jeeves and the Mating Season (Jeeves #9),P.G. Wodehouse/Jonathan Cecil,4.26,1572703199,9781572703193,eng,7,109,22,4/7/2003,AudioGO,five star
7743,29907,Republic,Plato/Benjamin Jowett/Elizabeth Watson Scharff...,3.94,1593080972,9781593080976,eng,496,392,33,8/1/2005,Barnes Noble Classics,four star
6416,24154,Judy Moody Predicts the Future (Judy Moody #4),Megan McDonald/Peter H. Reynolds,4.07,0763623431,9780763623432,eng,160,93,13,2/3/2005,Candlewick Press,five star
2224,7965,Homebody,Orson Scott Card,3.39,0061093998,9780061093999,eng,430,2707,167,1/25/1999,HarperTorch,four star
5655,21016,The Secret City (Pyrates #1),Chris Archer,4.11,0439368510,9780439368513,eng,184,231,17,2/1/2003,Scholastic Paperbacks,five star
9077,35402,Star Wars: The Ultimate Visual Guide,Daniel Wallace/Ryder Windham,4.33,0756630525,9780756630522,eng,160,945,17,3/19/2007,DK Children,five star
8633,33198,Romeo and Juliet,William Shakespeare/René Weis,3.74,1903436915,9781903436912,eng,350,219,13,7/15/2012,Bloomsbury Arden Shakespeare,four star
10709,43681,A Garden Of Vipers (Carson Ryder #3),Jack Kerley,3.96,0451412338,9780451412331,en-US,400,562,37,2/6/2007,Onyx,four star
9317,36681,Drop The Rock: Removing Character Defects - St...,Bill Pittman/Todd W. Hall/Sara S./Todd W.,4.44,1592851614,9781592851614,en-US,132,955,41,2/11/2005,Hazelden Publishing,five star


In [25]:
features = model[['language_code', 'rating_catergory', 'average_rating']]

In [26]:
features.average_rating = features.average_rating.astype(float)

C:\Users\saaim\AppData\Local\Temp\ipykernel_6260\418524463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.average_rating = features.average_rating.astype(float)


In [27]:
features.head()

,language_code,rating_catergory,average_rating
0,eng,five star,4.57
1,eng,five star,4.49
2,eng,five star,4.42
3,eng,five star,4.56
4,eng,five star,4.78


In [28]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   language_code     11123 non-null  object 
 1   rating_catergory  11123 non-null  object 
 2   average_rating    11123 non-null  float64
dtypes: float64(1), object(2)
memory usage: 260.8+ KB


In [38]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder(sparse=False)

In [42]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['language_code','rating_catergory']), sparse_threshold=0
)

In [43]:
features1 = column_trans.fit_transform(features)

In [56]:
features2 = pd.concat([pd.DataFrame(features1), features['average_rating']], axis=1)

In [57]:
type(features2)

pandas.core.frame.DataFrame

In [58]:
features2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   0               11123 non-null  float64
 1   1               11123 non-null  float64
 2   2               11123 non-null  float64
 3   3               11123 non-null  float64
 4   4               11123 non-null  float64
 5   5               11123 non-null  float64
 6   6               11123 non-null  float64
 7   7               11123 non-null  float64
 8   8               11123 non-null  float64
 9   9               11123 non-null  float64
 10  10              11123 non-null  float64
 11  11              11123 non-null  float64
 12  12              11123 non-null  float64
 13  13              11123 non-null  float64
 14  14              11123 non-null  float64
 15  15              11123 non-null  float64
 16  16              11123 non-null  float64
 17  17              11123 non-null 

In [59]:
from sklearn.preprocessing import MinMaxScaler

In [61]:
min_max_scaler = MinMaxScaler()
features3=min_max_scaler.fit_transform(features2)

In [63]:
from sklearn import neighbors

In [64]:
model= neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
model.fit(features3)
dist, idlist = model.kneighbors(features3)

In [66]:
pd.DataFrame(dist).head()

,0,1,2,3,4,5
0,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00
